In [9]:
import sys


sys.path.append("../src/")

In [10]:
from sennet.core.mmap_arrays import read_mmap_array, create_mmap_array
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import cc3d

In [21]:
# path = Path("/home/clay/research/kaggle/sennet/data_dumps/predicted/for_rg/kidney_3_sparse/chunk_00")
path = Path("/home/clay/research/kaggle/sennet/data_dumps/predicted/for_rg/kidney_2/chunk_00")

In [22]:
seed_path = path / "seed"
out_path = path / "out"

In [23]:
mean_pred = read_mmap_array(path / "mean_prob")


In [24]:
out_mmap = create_mmap_array(out_path, shape=mean_pred.data.shape, dtype=bool)
out_mmap.data[:] = False
out_mmap.data.flush()

# Picking Largest cc

In [ ]:
threshold = 0.03
thresholded_pred = np.ascontiguousarray(mean_pred.data > threshold)

In [ ]:
largest_cluster, label_n = cc3d.largest_k(
    thresholded_pred,
    k=1,
    connectivity=26,
    # delta=0,
    return_N=True,
)

In [ ]:
count = largest_cluster.sum()
label_count = thresholded_pred.sum()
print(f"{count}/{label_count}, {count/label_count}")

In [ ]:
seed_mmap = create_mmap_array(seed_path, shape=mean_pred.data.shape, dtype=bool)
seed_mmap.data[:] = largest_cluster
seed_mmap.data.flush()

In [ ]:
largest_cluster.shape

# Dust Heavily Thresholded

In [25]:
threshold = 0.5
thresholded_pred = np.ascontiguousarray(mean_pred.data > threshold)

In [26]:
dusted_pred = cc3d.dust(
    thresholded_pred,
    threshold=100,
    connectivity=26,
)

In [27]:
seed_mmap = create_mmap_array(seed_path, shape=mean_pred.data.shape, dtype=bool)
seed_mmap.data[:] = dusted_pred
seed_mmap.data.flush()

# Post Processing Watershed

In [18]:
out_mmap = read_mmap_array("/home/clay/research/kaggle/sennet/data_dumps/predicted/for_rg/kidney_3_sparse/chunk_00/out", mode="readwrite")
out = np.ascontiguousarray(out_mmap.data > 0.01)

In [19]:
out_cc, n_out_cc = cc3d.connected_components(
    out, 
    return_N=True,
    connectivity=26,
)

In [20]:
sizes = np.array([x.sum() for (_, x) in cc3d.each(
    out_cc, 
    binary=False,
    in_place=True,
)])
print(sizes)
print([f"{s*100:.3f}" for s in (sizes / sizes.sum())])

[8757805  185104  170727   14356    3530    5310]
['95.852', '2.026', '1.869', '0.157', '0.039', '0.058']


In [7]:
out_largest_cluster, _ = cc3d.largest_k(
    out,
    k=1,
    connectivity=26,
    return_N=True,
)

In [8]:
out_mmap.data[:] = out_largest_cluster
out_mmap.data.flush()